In [ ]:
Console.WriteLine(string.IsNullOrWhiteSpace(" "));
Console.WriteLine(string.IsNullOrWhiteSpace(string.Empty));

In [ ]:
DateTime? dob = Convert.ToDateTime("1977-07-17T04:00:00Z");

Console.WriteLine(dob?.ToString("yyy-MM-dd"))

In [ ]:
class Person
{
  public int? FirstName {get; set; }
  public int? LastName {get; set; }
}

var p = new Person();
var p1 = new Person();
if (p is not { FirstName: p1.FirstName, LastName: {} })
{
  Console.WriteLine("Is Null");
}
else
{
   Console.WriteLine("Not Null");
}

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Reflection;

// Working solution, how to handle switch cases
    public abstract class Enumeration<TEnum> where TEnum: class
    {
        private string Value { get; set; }

        protected Enumeration(string enumValue) => Value = enumValue;

        /// <summary>
        /// Convert to String
        /// </summary>
        /// <returns>Value of the enum</returns>
        public override string ToString() => Value;
        
        /// <summary>
        /// 
        /// </summary>
        /// <param name="obj"></param>
        /// <typeparam name="T"></typeparam>
        /// <returns></returns>
        public bool Equals<T>(object obj)
        {
            if (!(obj is Enumeration<TEnum> otherValue))
            {
                return false;
            }

            var typeMatches = GetType() == obj.GetType();
            var valueMatches = Value == otherValue.Value;

            return typeMatches && valueMatches;
        }
        
        
        /// <summary>
        /// 
        /// </summary>
        /// <param name="name">Name of the enum</param>
        /// <param name="enumValue"></param>
        /// <returns></returns>
        protected static bool TryGetEnum(string name, out TEnum? enumValue)
        {
            if (string.IsNullOrEmpty(name))
            {
                enumValue = null;
                return false;
            }
            name = name.ToUpper();
            var enums = GetAll<TEnum>().ToList();
            var enumDictionary = enums.ToDictionary(e => e.ToString());
            if (enumDictionary.ContainsKey(name))
            {
                enumValue = enumDictionary[name] as TEnum;
                return true;
            }
            enumValue = null;
            return false;
        }
        

        /// <summary>
        /// Get all the available types defined
        /// </summary>
        /// <typeparam name="T">Type of Enum class</typeparam>
        /// <returns></returns>
        private static IEnumerable<T> GetAll<T>() where T : TEnum =>
            typeof(T).GetFields(BindingFlags.Public |
                                BindingFlags.Static |
                                BindingFlags.DeclaredOnly)
                .Select(f => f.GetValue(null))
                .Cast<T>();
        
    }
}

 public sealed class EventName: Enumeration<EventName>
    {
       
        public static readonly EventName Insert = new EventName("INSERT");
        public static readonly EventName Modify = new EventName("MODIFY");
        public static readonly EventName Remove = new EventName("REMOVE");

        private EventName(string eventName): base(eventName)
        {
           
        }
        
        public static implicit operator string?(EventName? enumValue)
        {
            return enumValue?.ToString();
        }
        
        public static implicit operator EventName?(string name)
        {
            return TryGetEnum(name, out var enumValue) ? enumValue : null;
        }
    }

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;

public class MaritalStatusTypeConverter: JsonConverter<MaritalStatusType>
{
    public override MaritalStatusType Read(ref Utf8JsonReader reader,
            Type typeToConvert,
            JsonSerializerOptions options) {
                var strValue = reader.GetString();
                MaritalStatusType status =strValue;
                return status;
            }

    public override void Write(Utf8JsonWriter writer,
            MaritalStatusType maritalStatus,
            JsonSerializerOptions options) =>
                writer.WriteStringValue($"{maritalStatus.ToString()}");
}



[JsonConverter(typeof(MaritalStatusTypeConverter))]
public sealed class MaritalStatusType
{
    readonly string maritalStatus;
   
    public static readonly MaritalStatusType Single = new MaritalStatusType("Single");
    public static readonly MaritalStatusType Married = new MaritalStatusType("Married");
    public static readonly MaritalStatusType Widowed = new MaritalStatusType("Widowed");

    static Dictionary<string, MaritalStatusType> allowedValues = new Dictionary<string, MaritalStatusType>()
    {
        {Single.maritalStatus.ToUpper(), Single},
        {Married.maritalStatus.ToUpper(), Married},
        {Widowed.maritalStatus.ToUpper(), Widowed}
    };
    private MaritalStatusType(string value)
    {
        maritalStatus = value;
    }

    public static bool Has(string value)
    {
        if (allowedValues == null || allowedValues.Count == 0)
        {
            throw new Exception("Allowed Values is null or empty");
        }
    
       if (allowedValues.ContainsKey(value.ToUpper()))
        {
            return true;
        }
        return false;
    }

    public override string ToString()
    {
        return maritalStatus;
    }

    public static implicit operator string(MaritalStatusType value){
        return value.maritalStatus;
    }

    public static implicit operator MaritalStatusType(string value){
        if (!Has(value))
        {
            throw new Exception($"{value} is not assignable to MaritalStatusType");
        }
        return allowedValues[value.ToUpper()];
    }
}


/*----------------------------------------------------------------
    Using the type is done here
------------------------------------------------------------------*/

// What capabilities does this provide
// 1. Self Healing
Console.WriteLine("1. Self Healing");

MaritalStatusType SingleLowerCase = "single";
MaritalStatusType SingleUpperCase = "SINGLE";
MaritalStatusType Single = MaritalStatusType.Single;

Console.Write("\tAll lower case letter string assigned, ");
Console.WriteLine(SingleLowerCase);
Console.Write("\tAll Upper case letter string assigned, ");
Console.WriteLine(SingleUpperCase);
Console.Write("\tEnum style value assigned, ");
Console.WriteLine(Single);

//2. Equality Comparison
Console.WriteLine("\n2. Equality Comparison");
var counter = 0;
MaritalStatusType maritalStatus = MaritalStatusType.Single;
do{
    if (maritalStatus == MaritalStatusType.Single)
    {
        Console.WriteLine("\t:( I'm Single");
    } else 
    {
        Console.WriteLine("\t:) I'm Married");
    }
    maritalStatus = MaritalStatusType.Married;

}while(counter++ < 1);

//3. Safety
Console.WriteLine("\n3. Safety");
try
{
    MaritalStatusType status = "Something else";
}
catch(Exception ex)
{
    Console.WriteLine($"\tException: {ex.Message}");
}

//4. Additional Checking
Console.WriteLine("\n4. Checking");
if (!MaritalStatusType.Has("Separated")){
    Console.WriteLine("\tNope Separated is not in this type");
}

if (MaritalStatusType.Has("Married")){
    Console.WriteLine("\tYes Married is in the type");
}

//5. Serialization
Console.WriteLine("\n5. Serialization");
Console.WriteLine($"\t{JsonSerializer.Serialize(SingleLowerCase)}");

class Person{
    public string Name{get; init;}
    public MaritalStatusType MaritalStatus {get; init;}
}
var p = new Person() { Name="Bipin", MaritalStatus ="single" };
Console.WriteLine($"\t{JsonSerializer.Serialize(p)}");

//6. Deserialization
Console.WriteLine("\n6. Deserialization");
var jsonString = "{\"Name\":\"Bipin\",\"MaritalStatus\":\"Single\"}";
p = JsonSerializer.Deserialize<Person>(jsonString);
Console.WriteLine($"\t{p.MaritalStatus}");
